<a href="https://colab.research.google.com/github/leechungpa/Financial-Engineering-intro/blob/master/final_report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural Network 모형을 활용하여, Put-Call Parity 에 따른 이자율 예측

![](https://raw.githubusercontent.com/leechungpa/FinancialEngineering-intro/master/docs/Put-call%20Parity.png)

Put-call parity 공식에 따르면 $c-p=S_{0}-K/r$ 이 성립한다. 즉 현재시점에서 c와 p와 S0와 K를 알 수 있기에 차익거래가 존재하지 않는다면 사람들이 생각하는 이자율을 예측할 수 있다. 그래서 우리는 이 공식을 이용해 데이터를 만들어 보고, 해당데이터를 이용해 이자율을 예측하는 모형을 만들어 보고자 한다.

데이터를 만들 때 2가지의 방식으로 만들고자 한다. 먼저 정확한 공식에 따라 데이터를 만든 경우이다. 그리고 두번째는 임의의 오차를 준 경우이다. 우리는 공식에 따라 $S_{0}$를 계산해 데이터를 만들 것인데, 오차를 임의의 0.8~1.2 값을 배하여 구하였다.

또한 사람들이 생각하는 이자율은 다 다를 수 있으나, 특정한 값 근처에 몰릴 것이라고 판단해, 특정값(현 기준금리인 0.75)에 근접한 분포를 이용해 put-call parity 공식을 활용해 10개의 데이터 test set을 만들고, 만든 모형이 0.75에 근접한 값을 보이는지 확인해 보고자 한다.

In [1]:
from random import *
import pandas as pd
import tensorflow as tf

## 데이터 생성

In [2]:
# put-call parity 공식
def put_call_parity(c,p,K,r):
  value = c - p + K/r
  return value

# put-call parity 공식 및 오차 반영
def put_call_parity_with_error(c,p,K,r,error = 0.2):
  value = put_call_parity(c,p,K,r)*uniform(1-error, 1+error)
  return value

In [41]:
# Data Frame 생성
pcp_df = pd.DataFrame(columns=['c', 'p', 's', 'K', 'r'])

In [42]:
i=0
n=10 # 만들 데이터의 행의 수
while i != n :
  c = randint(20,200)
  p = randint(20,200)
  K = randint(20,200)
  r = random()+0.5
  if c > p :
    pcp_df.loc[i]=[c, p, put_call_parity(c,p,K,r), K, r]
    i=i+1
    print(c, p, put_call_parity(c,p,K,r), K, r)

99 50 117.6381498078763 68 0.9907026950804686
101 56 141.66446015691906 134 1.3862385387811893
165 48 225.6184217758956 102 0.9390672257275944
190 142 62.907816873465464 21 1.4086569601869783
188 93 260.88476097973285 172 1.036864380936199
171 122 97.85612900059073 40 0.8187304401361056
185 142 115.55740146391226 96 1.3230903817269055
160 67 161.3115533987893 72 1.0539944770349208
56 46 56.54152219676129 48 1.0313371315418687
188 93 449.7482575888318 195 0.5496855751325866


In [63]:
pcp_df

,c,p,s,K,r
0,99.0,50.0,117.638150,68.0,0.990703
1,101.0,56.0,141.664460,134.0,1.386239
2,165.0,48.0,225.618422,102.0,0.939067
3,190.0,142.0,62.907817,21.0,1.408657
4,188.0,93.0,260.884761,172.0,1.036864
5,171.0,122.0,97.856129,40.0,0.818730
6,185.0,142.0,115.557401,96.0,1.323090
7,160.0,67.0,161.311553,72.0,1.053994
8,56.0,46.0,56.541522,48.0,1.031337
9,188.0,93.0,449.748258,195.0,0.549686


In [64]:
pcp_X = pcp_df.loc[:,['c','p','s','K']]
pcp_Y = pcp_df.loc[:,'r']

## 모델생성

In [44]:
my_model = tf.keras.Sequential([
    tf.keras.layers.Dense(3, input_shape=[4],activation='relu'),
    tf.keras.layers.Dense(1)  #Dense 레이어의 경우 2번째 레이어 부터는 input_shape 필요 없음
]) 
my_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 3)                 9         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 4         
Total params: 13
Trainable params: 13
Non-trainable params: 0
_________________________________________________________________


In [69]:
my_model.compile(
    loss = 'mse',
    optimizer = 'adam'
)
my_model.fit(pcp_X, pcp_Y ,epochs=100,verbose=False) # vaerbose하면 결과를 간단히 보여줌

Epoch 1/100


ValueError: in user code:

    C:\Users\User\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:571 train_function  *
        outputs = self.distribute_strategy.run(
    C:\Users\User\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\User\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\User\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\User\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:531 train_step  **
        y_pred = self(x, training=True)
    C:\Users\User\anaconda3\lib\site-packages\tensorflow\python\keras\engine\base_layer.py:886 __call__
        self.name)
    C:\Users\User\a

In [ ]:

my_model.predict([[0.5,3]])

my_model.get_weights()

